In [1]:
!git clone https://github.com/binli123/dsmil-wsi.git

Cloning into 'dsmil-wsi'...
remote: Enumerating objects: 648, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 648 (delta 139), reused 130 (delta 118), pack-reused 459 (from 1)
Receiving objects: 100% (648/648), 49.23 MiB | 29.57 MiB/s, done.
Resolving deltas: 100% (357/357), done.


In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [3]:
%cd /content/dsmil-wsi
!conda env create -f env.yml
!conda run -n dsmil pip install torch
!conda run -n dsmil pip install torchvision


/content/dsmil-wsi
Channels:
 - anaconda
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



tesseract-5.2.0      | 171.4 MB  | :   0% 0/1 [00:00<?, ?it/s]
mysql-8.4.0          | 67.1 MB   | :   0% 0/1 [00:00<?, ?it/s]

python-3.13.5        | 42.8 MB   | :   0% 0/1 [00:00<?, ?it/s]


opencv-4.10.0        | 36.8 MB   | :   0% 0/1 [00:00<?, ?it/s]



icu-73.1             | 27.7 MB   | :   0% 0/1 [00:00<?, ?it/s]




scipy-1.15.3         | 26.6 MB   | :   0% 0/1 [00:00<?, ?it/s]





qtbase-6.7.3         | 17.9 MB   | :   0% 0/1 [00:00<?, ?it/s]






qtdeclarative-6.7.3  | 17.5 MB   | :   0% 0/1 [00:00<?, ?it/s]







pandas-2.2.3         | 16.5 MB   | :   0% 0/1 [00:00<?, ?it/s]








scikit-learn-1.6.1   | 12.7 MB   | :   0% 0/1 [00:00<?, ?it/s]









scik

In [ ]:
!conda run -n dsmil conda list


#### With pre-extracted features (UNI: https://huggingface.co/MahmoodLab/UNI)

In [ ]:
%cd /content/dsmil-wsi

import os
import zipfile

# URL for the dataset
url = "https://zenodo.org/records/15547611/files/datasetUnified_PT.zip?download=1"

# Download the file using wget
!wget -O MLiA_features.zip "$url"

# Define the extraction path
extract_path = './'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile('MLiA_features.zip', 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the contents of the extracted folder
extracted_files = os.listdir(extract_path)
print("Extracted files:", extracted_files)

/content/dsmil-wsi
--2025-06-26 12:27:45--  https://zenodo.org/records/15547611/files/datasetUnified_PT.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2005901570 (1.9G) [application/octet-stream]
Saving to: ‘MLiA_features.zip’

MLiA_features.zip   100%[===================>]   1.87G  18.7MB/s    in 1m 49s  

2025-06-26 12:29:35 (17.5 MB/s) - ‘MLiA_features.zip’ saved [2005901570/2005901570]

Extracted files: ['tcga-download', 'README.md', 'results_features', 'MLiA_features.zip', 'train_mil.py', 'compute_feats.py', 'testing_c16.py', 'deepzoom_tiler.py', 'test_crop_single.py', 'init.pth', '.gitignore', 'thumbnails', 'testing_tcga.py', 'simclr', 'dsmil.py', 'env.yml', 'train_tcga.py', 'example_aggregator_weights', '.git', 'LICENSE', 'download.py', 'attention_map.py']


In [ ]:
%cd /content/dsmil-wsi/results_features/h5_files
import h5py
with h5py.File('M-1.h5', 'r') as f:
    features = f['features'][:]  # or f['data'][:] depending on your key
print(features.shape)  # (num_patches, feature_dim)

%cd /content/dsmil-wsi/results_features/pt_files/
import torch
features = torch.load('M-1.pt')
print(features.shape)  # (num_patches, feature_dim)



/content/dsmil-wsi/results_features/h5_files
(19043, 1024)
/content/dsmil-wsi/results_features/pt_files
torch.Size([19043, 1024])


In [ ]:
import h5py
import pandas as pd
import os

h5_dir = '/content/dsmil-wsi/results_features/h5_files'

# Get a list of all .h5 files in the directory
all_files = os.listdir(h5_dir)
h5_files = [f for f in all_files if f.endswith('.h5')]

# Iterate through each .h5 file and convert to CSV
for h5_filename in h5_files:
    h5_file_path = os.path.join(h5_dir, h5_filename)
    csv_filename = os.path.splitext(h5_filename)[0] + '.csv'
    csv_output_path = os.path.join(h5_dir, csv_filename)

    try:
        # Read data from the HDF5 file
        with h5py.File(h5_file_path, 'r') as f:
            # Assuming your data is stored under the key 'features'
            features_data = f['features'][:]

        # Convert the numpy array to a pandas DataFrame
        df = pd.DataFrame(features_data)

        # Add a patch number column (starting from 1)
        #df.insert(0, 'patch_number', range(1, 1 + len(df)))

        # Write the DataFrame to a CSV file
        df.to_csv(csv_output_path, index=False, header=False)

        print(f"Successfully converted {h5_filename} to {csv_filename}")

    except Exception as e:
        print(f"Error converting {h5_filename}: {e}")

Successfully converted M-105.h5 to M-105.csv
Successfully converted M-111.h5 to M-111.csv
Successfully converted M-109.h5 to M-109.csv
Successfully converted M-10.h5 to M-10.csv
Successfully converted M-108.h5 to M-108.csv
Successfully converted M-103.h5 to M-103.csv
Successfully converted M-30.h5 to M-30.csv
Successfully converted M-112.h5 to M-112.csv
Successfully converted M-11.h5 to M-11.csv
Successfully converted M-1.h5 to M-1.csv
Successfully converted M-114.h5 to M-114.csv
Successfully converted M-86.h5 to M-86.csv
Successfully converted M-110.h5 to M-110.csv
Successfully converted M-100.h5 to M-100.csv
Successfully converted M-104.h5 to M-104.csv
Successfully converted M-87.h5 to M-87.csv
Successfully converted M-113.h5 to M-113.csv
Successfully converted M-32.h5 to M-32.csv
Successfully converted M-65.h5 to M-65.csv
Successfully converted M-121.h5 to M-121.csv
Successfully converted M-24.h5 to M-24.csv
Successfully converted M-101.h5 to M-101.csv


In [ ]:
import os
import pandas as pd

label_dict = {
    'M-1': 'B', 'M-104': 'B', 'M-105': 'B', 'M-108': 'B', 'M-112': 'B',
    'M-121': 'B', 'M-24': 'B', 'M-30': 'B', 'M-32': 'B',
    'M-10': 'E', 'M-100': 'E', 'M-103': 'E', 'M-109': 'E', 'M-11': 'E',
    'M-110': 'E', 'M-111': 'E', 'M-113': 'E',
    'M-101': 'S', 'M-114': 'S', 'M-65': 'S', 'M-86': 'S', 'M-87': 'S'
}

label_map = {'B': 0, 'S': 1, 'E': 2}

csv_dir = '/content/dsmil-wsi/results_features/h5_files'
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

csv_data = []

for f in csv_files:
    slide_id = os.path.splitext(f)[0]
    if slide_id in label_dict:
        label_int = label_map[label_dict[slide_id]]
        csv_data.append([os.path.join(csv_dir, f), label_int])

# Save without header and without index
output_path = '/content/dsmil-wsi/results_features/MLiA_dataset.csv'
pd.DataFrame(csv_data).to_csv(output_path, index=False, header=False)

In [ ]:
## to train paste into train_tcga.py the content of train_MLiA.py
%cd /content/dsmil-wsi
!MPLBACKEND=Agg conda run -n dsmil python train_tcga.py --dataset=MLiA_dataset --num_classes=3 --feats_size=1024 --num_epochs=300  --dropout_patch=0.50 --lr=1e-5 --stop_epochs=100 --eval_scheme=5-fold-cv --split=0.2


Output streaming troncato alle ultime 5000 righe.
 Training bag [6/10] bag loss: 0.6743
 Training bag [7/10] bag loss: 0.6685
 Training bag [8/10] bag loss: 0.6706
 Training bag [9/10] bag loss: 0.6688
 Testing bag [0/11] bag loss: 0.7149
 Testing bag [1/11] bag loss: 0.7111
 Testing bag [2/11] bag loss: 0.6692
 Testing bag [3/11] bag loss: 0.6701
 Testing bag [4/11] bag loss: 0.6685
 Testing bag [5/11] bag loss: 0.6645
 Testing bag [6/11] bag loss: 0.6714
 Testing bag [7/11] bag loss: 0.7140
 Testing bag [8/11] bag loss: 0.6694
 Testing bag [9/11] bag loss: 0.6676
 Testing bag [10/11] bag loss: 0.6726ROC AUC score: 0.6785714285714285
ROC AUC score: 1.0
ROC AUC score: 0.6785714285714286

 Epoch [50/300] train loss: 0.6798 test loss: 0.6812, average score: 0.5455, AUC: class-0>>0.6785714285714285|class-1>>1.0|class-2>>0.6785714285714286

 Training bag [0/10] bag loss: 0.6724
 Training bag [1/10] bag loss: 0.6714
 Training bag [2/10] bag loss: 0.6728
 Training bag [3/10] bag loss: 0.7164

In [ ]:
## to train paste into train_tcga.py the content of train_MLiA.py
%cd /content/dsmil-wsi
!MPLBACKEND=Agg conda run -n dsmil python train_tcga.py --dataset=MLiA_dataset --num_classes=3 --feats_size=1024 --num_epochs=300  --dropout_patch=0.60 --lr=1e-4 --weight_decay=1e-4 --stop_epochs=100 --eval_scheme=5-fold-cv-custom --split=0.2


5-fold-cv-custom
Creating intermediate training files.
5-fold-cv

🌀 Starting CV Fold 0

 Training bag [0/11] bag loss: 0.7013
 Training bag [1/11] bag loss: 0.7129
 Training bag [2/11] bag loss: 0.6979
 Training bag [3/11] bag loss: 0.6956
 Training bag [4/11] bag loss: 0.7158
 Training bag [5/11] bag loss: 0.7027
 Training bag [6/11] bag loss: 0.7098
 Training bag [7/11] bag loss: 0.6995
 Training bag [8/11] bag loss: 0.7048
 Training bag [9/11] bag loss: 0.6924
 Training bag [10/11] bag loss: 0.6937
 Testing bag [0/5] bag loss: 0.7136
 Testing bag [1/5] bag loss: 0.6902
 Testing bag [2/5] bag loss: 0.6969
 Testing bag [3/5] bag loss: 0.6977
 Testing bag [4/5] bag loss: 0.6979ROC AUC score: 0.5
ROC AUC score: 0.75
ROC AUC score: 0.33333333333333337

 Epoch [1/300] train loss: 0.7024 test loss: 0.6993, average score: 0.2000, AUC: class-0>>0.5|class-1>>0.75|class-2>>0.33333333333333337
Best model saved at: weights/20250626/fold_0_19.pth
Best thresholds ===>>> class-0>>0.5126194953918457

#### With SIMclr feature extraction

In [26]:
%cd /content/dsmil-wsi

import os
import zipfile

# URL for the dataset
url = "https://zenodo.org/records/15700269/files/datasetWSI.zip?download=1"

# Define the download and extraction path
download_path = 'datasetWSI.zip'
extract_path = 'datasetWSI/'

# Download the file using wget
!wget -O "$download_path" "$url"

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(download_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Downloaded and extracted to {extract_path}")

# List the contents of the extracted folder
extracted_files = os.listdir(extract_path)
print("Extracted files:", extracted_files)

/content/dsmil-wsi
--2025-06-27 07:58:58--  https://zenodo.org/records/15700269/files/datasetWSI.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.48.194, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11982067512 (11G) [application/octet-stream]
Saving to: ‘datasetWSI.zip’

datasetWSI.zip      100%[===================>]  11.16G  2.63MB/s    in 62m 42s 

2025-06-27 09:01:41 (3.04 MB/s) - ‘datasetWSI.zip’ saved [11982067512/11982067512]

Downloaded and extracted to datasetWSI/
Extracted files: ['ndpi_files']


In [27]:

import os

old_path = '/content/dsmil-wsi/datasetWSI'
new_path = '/content/dsmil-wsi/WSI'

os.rename(old_path, new_path)
print(f"Renamed: {old_path} → {new_path}")



Renamed: /content/dsmil-wsi/datasetWSI → /content/dsmil-wsi/WSI


In [28]:
!conda run -n dsmil pip install openslide-python openslide-bin


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 17.2 MB/s eta 0:00:00




In [31]:
# PATCH CROPPING
%cd /content/dsmil-wsi
!MPLBACKEND=Agg conda run -n dsmil python deepzoom_tiler.py --magnifications 0 -b 10 -d ndpi_files --workers 4 --slide_format ndpi --tile_size 256


Output streaming troncato alle ultime 5000 righe.
 Patch [3128/4865]
 Patch [3129/4865]
 Patch [3130/4865]
 Patch [3131/4865]
 Patch [3132/4865]
 Patch [3133/4865]
 Patch [3134/4865]
 Patch [3135/4865]
 Patch [3136/4865]
 Patch [3137/4865]
 Patch [3138/4865]
 Patch [3139/4865]
 Patch [3140/4865]
 Patch [3141/4865]
 Patch [3142/4865]
 Patch [3143/4865]
 Patch [3144/4865]
 Patch [3145/4865]
 Patch [3146/4865]
 Patch [3147/4865]
 Patch [3148/4865]
 Patch [3149/4865]
 Patch [3150/4865]
 Patch [3151/4865]
 Patch [3152/4865]
 Patch [3153/4865]
 Patch [3154/4865]
 Patch [3155/4865]
 Patch [3156/4865]
 Patch [3157/4865]
 Patch [3158/4865]
 Patch [3159/4865]
 Patch [3160/4865]
 Patch [3161/4865]
 Patch [3162/4865]
 Patch [3163/4865]
 Patch [3164/4865]
 Patch [3165/4865]
 Patch [3166/4865]
 Patch [3167/4865]
 Patch [3168/4865]
 Patch [3169/4865]
 Patch [3170/4865]
 Patch [3171/4865]
 Patch [3172/4865]
 Patch [3173/4865]
 Patch [3174/4865]
 Patch [3175/4865]
 Patch [3176/4865]
 Patch [3177/4865]


In [48]:
import torch
import gc

def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()    # Libera la cache della GPU
        torch.cuda.ipc_collect()    # Raccoglie la memoria non più usata dalla GPU
    gc.collect()                    # Libera la RAM

free_memory()

In [45]:
!conda run -n dsmil pip install tensorboard

In [53]:
  # Train an embedder SIMClr
  %cd /content/dsmil-wsi/simclr
  !MPLBACKEND=Agg conda run -n dsmil python run.py --dataset=ndpi_files

/content/dsmil-wsi/simclr
Please install apex for mixed precision training from: https://github.com/NVIDIA/apex
Running on: cuda
Feature extractor: resnet18
ResNetSimCLR(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (1): BasicBlock(
        (co

In [ ]:
import os
import zipfile
import requests
import json

# Your Zenodo access token here
ACCESS_TOKEN = 'uVSb7icJqT9efPM71KYgviJ50r7eML9ynei2q7hDkedVlFrf8fBsr9lFaJ3O'  # <-- Put your token here

# Folder you want to zip and upload
folder_dest = '/content/dsmil-wsi/WSI/ndpi_files/single'

# 1. Create ZIP archive of folder, maintaining structure
def create_zip_archive(folder_path, zip_name=None):
    if zip_name is None:
        zip_name = f"{os.path.basename(folder_path)}.zip"

    print(f"🗜️ Creating ZIP archive: {zip_name}")
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arc_name = os.path.relpath(file_path, os.path.dirname(folder_path))
                zipf.write(file_path, arc_name)
                print(f"  📁 Added: {arc_name}")

    zip_size = os.path.getsize(zip_name)
    print(f"✅ Archive created: {zip_name} ({zip_size/1024/1024:.2f} MB)")
    return zip_name


# 2. Create a new Zenodo deposition (dataset)
def create_deposition(title):
    url = 'https://zenodo.org/api/deposit/depositions'
    headers = {"Content-Type": "application/json"}
    params = {'access_token': ACCESS_TOKEN}

    data = {
        'metadata': {
            'title': title,
            'upload_type': 'dataset',
            'description': 'Working on DSMIL patch extraction dataset',
            'creators': [{'name': 'Raf-Tony-Luca'}]
        }
    }

    response = requests.post(url, params=params, data=json.dumps(data), headers=headers)
    response.raise_for_status()
    return response.json()


# 3. Upload the ZIP file to the Zenodo bucket URL
def upload_file(deposition_id, file_path):
    url = f'https://zenodo.org/api/deposit/depositions/{deposition_id}'
    params = {'access_token': ACCESS_TOKEN}
    r = requests.get(url, params=params)
    r.raise_for_status()
    bucket_url = r.json()["links"]["bucket"]

    filename = os.path.basename(file_path)
    with open(file_path, "rb") as fp:
        r = requests.put(f"{bucket_url}/{filename}", data=fp, params=params)
        r.raise_for_status()
    print(f"✅ File {filename} uploaded successfully.")
    return r.json()


# 4. Publish the deposition (make it public)
def publish_deposition(deposition_id):
    url = f'https://zenodo.org/api/deposit/depositions/{deposition_id}/actions/publish'
    params = {'access_token': ACCESS_TOKEN}
    r = requests.post(url, params=params)
    r.raise_for_status()
    print("✅ Deposition published successfully.")
    return r.json()


# --- Main flow ---

# Change dir if you want, here just for context
os.chdir('/content/dsmil-wsi')

zip_filename = 'WSI_single.zip'

# Create ZIP archive
zip_path = create_zip_archive(folder_dest, zip_filename)

# Create deposition
print("Creating Zenodo deposition...")
deposition = create_deposition("DSMIL_patch_extraction_single")
deposition_id = deposition['id']

# Upload file
print(f"Uploading file to deposition ID: {deposition_id} ...")
upload_result = upload_file(deposition_id, zip_path)

# Publish dataset
print("Publishing dataset on Zenodo...")
publication = publish_deposition(deposition_id)

print(f"🎉 Dataset published! DOI: {publication['doi']}")
print(f"🌐 URL: {publication['links']['record_html']}")


Output streaming troncato alle ultime 5000 righe.
  📁 Added: single/S/M-86/60_47.jpeg
  📁 Added: single/S/M-86/42_154.jpeg
  📁 Added: single/S/M-86/49_88.jpeg
  📁 Added: single/S/M-86/35_100.jpeg
  📁 Added: single/S/M-86/52_81.jpeg
  📁 Added: single/S/M-86/111_164.jpeg
  📁 Added: single/S/M-86/160_122.jpeg
  📁 Added: single/S/M-86/60_40.jpeg
  📁 Added: single/S/M-86/24_148.jpeg
  📁 Added: single/S/M-86/182_152.jpeg
  📁 Added: single/S/M-86/15_74.jpeg
  📁 Added: single/S/M-86/133_76.jpeg
  📁 Added: single/S/M-86/82_84.jpeg
  📁 Added: single/S/M-86/10_48.jpeg
  📁 Added: single/S/M-86/131_60.jpeg
  📁 Added: single/S/M-86/120_154.jpeg
  📁 Added: single/S/M-86/104_111.jpeg
  📁 Added: single/S/M-86/41_37.jpeg
  📁 Added: single/S/M-86/43_49.jpeg
  📁 Added: single/S/M-86/111_189.jpeg
  📁 Added: single/S/M-86/144_167.jpeg
  📁 Added: single/S/M-86/79_113.jpeg
  📁 Added: single/S/M-86/164_98.jpeg
  📁 Added: single/S/M-86/115_120.jpeg
  📁 Added: single/S/M-86/60_36.jpeg
  📁 Added: single/S/M-86/66

In [55]:
from PIL import Image

path = '/content/dsmil-wsi/WSI/ndpi_files/single/E/M-111/65_113.jpeg'
try:
    img = Image.open(path)
    img.show()
except Exception as e:
    print("Cannot open image:", e)


Cannot open image: cannot identify image file '/content/dsmil-wsi/WSI/ndpi_files/single/E/M-111/65_113.jpeg'
